# Setup

## Download Data

In [218]:
startDate = '2025-06-30' #inclusive
endDate = '2025-07-01' #not inclusive
interval = '1m'

In [219]:
import yfinance as yf
import pandas as pd
import numpy as np
# Get a list of S&P 500 tickers from Wikipedia
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500 = pd.read_html(sp500_url)[0]
tickers = sp500['Symbol'].tolist()
# Handle tickers with periods (e.g., BRK.B) for yfinance
tickers = [t.replace('.', '-') for t in tickers]
# Download historical data for tickers
data = yf.download(tickers, start=startDate, end=endDate, interval=interval, auto_adjust=True)["Close"]
data


[*********************100%***********************]  503 of 503 completed


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-06-30 13:30:00+00:00,118.989998,201.917496,182.279999,134.559998,134.615005,90.360001,297.105011,387.119995,237.475006,51.904999,...,302.505005,25.900000,94.540001,67.419998,108.519997,127.830002,148.244995,91.360001,311.570007,155.979996
2025-06-30 13:31:00+00:00,118.824997,201.554993,182.500000,134.270004,134.125000,90.254997,296.790009,387.049988,237.259995,51.933399,...,302.329987,25.700001,94.180000,67.434998,108.356796,127.910004,147.660004,90.949997,NaN,155.945007
2025-06-30 13:32:00+00:00,NaN,201.389999,NaN,134.509995,NaN,90.794998,NaN,386.364990,236.199997,NaN,...,303.234985,NaN,93.610001,67.565002,NaN,NaN,NaN,NaN,307.619995,NaN
2025-06-30 13:33:00+00:00,117.985001,201.110001,183.410004,134.479996,134.910004,90.887497,297.549988,387.049988,236.335007,51.845001,...,304.334991,25.860001,93.750000,67.750000,108.539703,127.430000,148.119995,91.059998,307.600006,156.259995
2025-06-30 13:34:00+00:00,118.559998,200.895004,183.600006,134.500000,135.250000,90.830002,298.079987,387.149994,236.500000,51.770000,...,NaN,25.799999,93.738701,67.730003,108.625000,127.825600,148.339996,91.230003,NaN,156.214996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30 19:55:00+00:00,118.260002,205.645401,185.729996,132.335007,136.020004,91.089996,298.630005,387.154999,238.369995,52.805000,...,306.595001,25.660000,93.760002,68.059998,107.839996,129.425003,148.264999,91.169998,308.709991,155.759995
2025-06-30 19:56:00+00:00,118.120003,205.455002,185.830002,132.289993,136.009995,90.974998,298.640015,387.089996,238.324997,52.855000,...,306.575012,25.674999,93.690002,68.059998,107.870003,129.389999,148.324997,91.180000,308.570007,155.705002
2025-06-30 19:57:00+00:00,118.059998,205.695007,185.740005,132.440002,136.080002,91.089996,298.880005,387.339996,238.199997,52.855000,...,306.579987,25.715000,93.794998,68.095001,107.949997,129.490005,148.309998,91.160004,308.769989,155.750000


## Clean Data

In [221]:
# Count total NaNs per ticker
total_nans = data.isna().sum()
print("Total NaNs per ticker:")
print(total_nans[total_nans > 0])

Total NaNs per ticker:
Ticker
A        43
AAPL      3
ABBV      3
ABNB      2
ABT       2
       ... 
XYL      31
YUM       7
ZBH      28
ZBRA    212
ZTS       5
Length: 502, dtype: int64


### Deal With NaN Values

In [222]:
ACCEPTABLE_NAN_THRESHOLD = 0.9
# Remove any tickers that failed to download enough data
clean_data = data.dropna(axis=1, thresh=int(ACCEPTABLE_NAN_THRESHOLD * len(data)))
clean_data

Ticker,AAPL,ABBV,ABNB,ABT,ACN,ADBE,ADI,ADM,AEE,AEP,...,WMT,WSM,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-06-30 13:30:00+00:00,201.917496,182.279999,134.559998,134.615005,297.105011,387.119995,237.475006,51.904999,95.029999,102.480003,...,97.419998,164.684998,25.900000,94.540001,67.419998,108.519997,127.830002,148.244995,91.360001,155.979996
2025-06-30 13:31:00+00:00,201.554993,182.500000,134.270004,134.125000,296.790009,387.049988,237.259995,51.933399,NaN,102.129997,...,97.190002,164.000000,25.700001,94.180000,67.434998,108.356796,127.910004,147.660004,90.949997,155.945007
2025-06-30 13:32:00+00:00,201.389999,NaN,134.509995,NaN,NaN,386.364990,236.199997,NaN,NaN,102.169998,...,NaN,NaN,NaN,93.610001,67.565002,NaN,NaN,NaN,NaN,NaN
2025-06-30 13:33:00+00:00,201.110001,183.410004,134.479996,134.910004,297.549988,387.049988,236.335007,51.845001,95.279999,102.387604,...,97.470001,162.648407,25.860001,93.750000,67.750000,108.539703,127.430000,148.119995,91.059998,156.259995
2025-06-30 13:34:00+00:00,200.895004,183.600006,134.500000,135.250000,298.079987,387.149994,236.500000,51.770000,95.279999,102.389999,...,97.570000,162.955002,25.799999,93.738701,67.730003,108.625000,127.825600,148.339996,91.230003,156.214996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30 19:55:00+00:00,205.645401,185.729996,132.335007,136.020004,298.630005,387.154999,238.369995,52.805000,95.800003,103.764999,...,97.945000,163.059998,25.660000,93.760002,68.059998,107.839996,129.425003,148.264999,91.169998,155.759995
2025-06-30 19:56:00+00:00,205.455002,185.830002,132.289993,136.009995,298.640015,387.089996,238.324997,52.855000,95.849998,103.760002,...,97.867996,163.070007,25.674999,93.690002,68.059998,107.870003,129.389999,148.324997,91.180000,155.705002
2025-06-30 19:57:00+00:00,205.695007,185.740005,132.440002,136.080002,298.880005,387.339996,238.199997,52.855000,95.919998,103.800003,...,97.900002,163.255005,25.715000,93.794998,68.095001,107.949997,129.490005,148.309998,91.160004,155.750000


In [223]:
# Define trading hours in UTC (13:30 to 20:00) and generate valid weekdays only
full_days = pd.date_range(start=startDate, end=endDate, freq='B')  # 'B' = business days (Mon–Fri)
full_index = pd.DatetimeIndex([])
for day in full_days[:-1]:  # Exclude the last day since it's not inclusive
    trading_day_minutes = pd.date_range(
        start=day.strftime('%Y-%m-%d') + ' 13:30',
        end=day.strftime('%Y-%m-%d') + ' 20:00',
        freq='1min',
        tz='UTC'
    )
    full_index = full_index.append(trading_day_minutes)

# Reindex to include all valid trading minutes
clean_data = clean_data.reindex(full_index)

# Forward and backward fill for small gaps
clean_data.ffill(inplace=True, limit=2)
clean_data.bfill(inplace=True, limit=2)
clean_data

/var/folders/gs/kw3ckzg507vcmd706g45pv4w0000gn/T/ipykernel_10565/4065491124.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  full_index = full_index.append(trading_day_minutes)


Ticker,AAPL,ABBV,ABNB,ABT,ACN,ADBE,ADI,ADM,AEE,AEP,...,WMT,WSM,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
2025-06-30 13:30:00+00:00,201.917496,182.279999,134.559998,134.615005,297.105011,387.119995,237.475006,51.904999,95.029999,102.480003,...,97.419998,164.684998,25.900000,94.540001,67.419998,108.519997,127.830002,148.244995,91.360001,155.979996
2025-06-30 13:31:00+00:00,201.554993,182.500000,134.270004,134.125000,296.790009,387.049988,237.259995,51.933399,95.029999,102.129997,...,97.190002,164.000000,25.700001,94.180000,67.434998,108.356796,127.910004,147.660004,90.949997,155.945007
2025-06-30 13:32:00+00:00,201.389999,182.500000,134.509995,134.125000,296.790009,386.364990,236.199997,51.933399,95.029999,102.169998,...,97.190002,164.000000,25.700001,93.610001,67.565002,108.356796,127.910004,147.660004,90.949997,155.945007
2025-06-30 13:33:00+00:00,201.110001,183.410004,134.479996,134.910004,297.549988,387.049988,236.335007,51.845001,95.279999,102.387604,...,97.470001,162.648407,25.860001,93.750000,67.750000,108.539703,127.430000,148.119995,91.059998,156.259995
2025-06-30 13:34:00+00:00,200.895004,183.600006,134.500000,135.250000,298.079987,387.149994,236.500000,51.770000,95.279999,102.389999,...,97.570000,162.955002,25.799999,93.738701,67.730003,108.625000,127.825600,148.339996,91.230003,156.214996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30 19:56:00+00:00,205.455002,185.830002,132.289993,136.009995,298.640015,387.089996,238.324997,52.855000,95.849998,103.760002,...,97.867996,163.070007,25.674999,93.690002,68.059998,107.870003,129.389999,148.324997,91.180000,155.705002
2025-06-30 19:57:00+00:00,205.695007,185.740005,132.440002,136.080002,298.880005,387.339996,238.199997,52.855000,95.919998,103.800003,...,97.900002,163.255005,25.715000,93.794998,68.095001,107.949997,129.490005,148.309998,91.160004,155.750000
2025-06-30 19:58:00+00:00,205.539993,185.774994,132.369995,136.100006,298.934998,387.290009,238.080002,52.845001,95.959999,103.824997,...,97.864998,163.389999,25.695000,93.735001,68.110001,107.859901,129.460007,148.250000,91.205002,155.830002
2025-06-30 19:59:00+00:00,205.119995,185.669998,132.300003,135.970001,298.769989,386.709991,238.029999,52.770000,95.989998,103.790001,...,97.754997,163.369995,25.680000,93.629997,68.080002,107.769997,129.350006,148.089996,91.190002,155.889999


In [224]:
print(clean_data.isna().sum().sort_values(ascending=False).head(10))  # Top columns with missing data
print(clean_data.isna().any(axis=1).sum())  # How many rows have NaNs

Ticker
LKQ     1
AAPL    0
MMC     0
NKE     0
NI      0
NFLX    0
NEM     0
NEE     0
NDAQ    0
NCLH    0
dtype: int64
1


In [240]:
# Drop any remaining NaN values (if any)
clean_data.dropna(inplace=True, axis=1, how='any')  # Drop columns with all NaN values
clean_data

Ticker,AAPL,ABBV,ABNB,ABT,ACN,ADBE,ADI,ADM,AEE,AEP,...,WMT,WSM,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
2025-06-30 13:30:00+00:00,201.917496,182.279999,134.559998,134.615005,297.105011,387.119995,237.475006,51.904999,95.029999,102.480003,...,97.419998,164.684998,25.900000,94.540001,67.419998,108.519997,127.830002,148.244995,91.360001,155.979996
2025-06-30 13:31:00+00:00,201.554993,182.500000,134.270004,134.125000,296.790009,387.049988,237.259995,51.933399,95.029999,102.129997,...,97.190002,164.000000,25.700001,94.180000,67.434998,108.356796,127.910004,147.660004,90.949997,155.945007
2025-06-30 13:32:00+00:00,201.389999,182.500000,134.509995,134.125000,296.790009,386.364990,236.199997,51.933399,95.029999,102.169998,...,97.190002,164.000000,25.700001,93.610001,67.565002,108.356796,127.910004,147.660004,90.949997,155.945007
2025-06-30 13:33:00+00:00,201.110001,183.410004,134.479996,134.910004,297.549988,387.049988,236.335007,51.845001,95.279999,102.387604,...,97.470001,162.648407,25.860001,93.750000,67.750000,108.539703,127.430000,148.119995,91.059998,156.259995
2025-06-30 13:34:00+00:00,200.895004,183.600006,134.500000,135.250000,298.079987,387.149994,236.500000,51.770000,95.279999,102.389999,...,97.570000,162.955002,25.799999,93.738701,67.730003,108.625000,127.825600,148.339996,91.230003,156.214996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30 19:56:00+00:00,205.455002,185.830002,132.289993,136.009995,298.640015,387.089996,238.324997,52.855000,95.849998,103.760002,...,97.867996,163.070007,25.674999,93.690002,68.059998,107.870003,129.389999,148.324997,91.180000,155.705002
2025-06-30 19:57:00+00:00,205.695007,185.740005,132.440002,136.080002,298.880005,387.339996,238.199997,52.855000,95.919998,103.800003,...,97.900002,163.255005,25.715000,93.794998,68.095001,107.949997,129.490005,148.309998,91.160004,155.750000
2025-06-30 19:58:00+00:00,205.539993,185.774994,132.369995,136.100006,298.934998,387.290009,238.080002,52.845001,95.959999,103.824997,...,97.864998,163.389999,25.695000,93.735001,68.110001,107.859901,129.460007,148.250000,91.205002,155.830002
2025-06-30 19:59:00+00:00,205.119995,185.669998,132.300003,135.970001,298.769989,386.709991,238.029999,52.770000,95.989998,103.790001,...,97.754997,163.369995,25.680000,93.629997,68.080002,107.769997,129.350006,148.089996,91.190002,155.889999


# Select Pairs

## Perform PCA

In [249]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Calculate log returns
returns = np.log(clean_data / clean_data.shift(1)).dropna()
# Scale log returns
X = returns.T
scaler = StandardScaler()
scaled_returns = scaler.fit_transform(X)
scaled_returns

array([[-0.12489073, -0.30108078, -0.60538155, ..., -1.08129556,
        -2.11168704,  0.        ],
       [ 0.6601675 ,  0.0671236 ,  1.22560283, ...,  0.50625515,
         0.21981201,  0.        ],
       [-0.21913858,  0.87003042, -0.26932936, ..., -0.70193655,
         0.27720543,  0.        ],
       ...,
       [-0.68876011,  0.0671236 ,  0.68954034, ..., -0.49282499,
        -0.59066625,  0.        ],
       [-0.83095906,  0.0671236 ,  0.14252781, ...,  1.02037096,
         0.85124877,  0.        ],
       [ 0.28620211,  0.0671236 ,  0.37526639, ...,  1.05411492,
         1.71668173,  0.        ]], shape=(352, 390))

In [250]:
# Perform PCA
pca = PCA(n_components=0.85)  # Percent variance explained threshold
pca.fit(X)
print(f"Number of components: {pca.n_components_}")
print(f"Variance explained by each eigenvector: {pca.explained_variance_ratio_}")
print(f"Total: {pca.explained_variance_ratio_.sum() * 100:.2f}%")

Number of components: 82
Variance explained by each eigenvector: [0.09935902 0.0780911  0.0515261  0.0403847  0.04028725 0.026587
 0.02347182 0.02154157 0.0208811  0.01845021 0.01800874 0.01684852
 0.01562876 0.01527224 0.01455518 0.0132015  0.01313054 0.01182131
 0.01142474 0.01073908 0.01018422 0.00950635 0.00931176 0.00904897
 0.00881441 0.00850473 0.00813755 0.00792354 0.00764429 0.0073637
 0.0072711  0.00698622 0.00656654 0.00651355 0.00629444 0.00604961
 0.00563292 0.00557297 0.00539307 0.00530345 0.00509633 0.00497783
 0.00485586 0.00475839 0.00461721 0.00446898 0.00432133 0.00423262
 0.00416001 0.00403296 0.00396383 0.00387865 0.0038253  0.00381261
 0.00374452 0.00366153 0.00362241 0.00353107 0.0034345  0.00335292
 0.00323647 0.00318769 0.00314347 0.00308233 0.00307057 0.00296177
 0.0029513  0.00289889 0.0028488  0.00276625 0.00270603 0.002658
 0.00263624 0.00258235 0.00254671 0.00252426 0.00242063 0.00239633
 0.00236119 0.00231692 0.00230507 0.00227437]
Total: 85.15%


### Clustering

In [ ]:
"""

from sklearn.cluster import KMeans

num_clusters = 25 # Can be changed
kmeans = KMeans(n_clusters=num_clusters, random_state=1)
kmeans.fit(X)

tickers = returns.columns
label_dict = {i: [] for i in range(num_clusters)}
for i, label in enumerate(kmeans.labels_):
    label_dict[label].append(tickers[i])
print("Clustered tickers:")
for label, tickers in label_dict.items():
    print(f"Cluster {label}: {', '.join(tickers)}")

"""

Clustered tickers:
Cluster 0: GEV
Cluster 1: APO, AXP, BAC, BK, C, CFG, COF, F, FI, FITB, GLW, GM, GPN, GS, HBAN, HIG, HLT, HPE, IVZ, JPM, KEY, KKR, MET, MS, MTB, NDAQ, NTRS, PNC, PRU, RF, SCHW, STT, STX, SYF, TFC, USB, WBD, WFC
Cluster 2: AEE, AEP, AES, AWK, CMS, CNP, D, DUK, ED, EIX, ES, ETR, EVRG, EXC, FE, LNT, NEE, NI, PEG, PPL, SO, SRE, WEC, XEL
Cluster 3: BBY, DLTR, EL, LVS, TGT, WSM
Cluster 4: FSLR
Cluster 5: ABNB, BX, COIN, FOXA, RCL
Cluster 6: ARE, BXP
Cluster 7: AAPL, ABBV, ABT, ACN, ADBE, AFL, AIG, AKAM, ALL, AME, AMGN, AMZN, APH, BA, BDX, BEN, BG, BKR, BMY, BRK-B, BRO, BSX, CAH, CB, CF, CHD, CI, CL, CLX, CMCSA, CME, CMG, CNC, COP, CPRT, CRM, CRWD, CSCO, CSX, CTRA, CVS, CVX, DASH, DG, DRI, DVN, EA, EBAY, ECL, ELV, EMR, EOG, EQT, EW, EXE, FANG, FAST, FIS, FTNT, GE, GEHC, GILD, HAL, HAS, HES, HON, HWM, IBM, ICE, IFF, IPG, JCI, JNJ, JNPR, K, KMI, KO, KR, KVUE, L, LLY, LYV, MA, MCD, MDT, META, MGM, MMC, MMM, MO, MOS, MPC, MRK, MSFT, NEM, NFLX, NRG, NUE, NWSA, OKE, OMC, ORLY, OTI

In [271]:
# Agglomerative clustering
VERBOSE = True  # Set to False to suppress verbose output
from sklearn.cluster import AgglomerativeClustering
num_clusters = 100 # Can be changed
clustering = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
clustering.fit_predict(X)
tickers = returns.columns
label_dict = {i: [] for i in range(num_clusters)}
for i, label in enumerate(clustering.labels_):
    label_dict[label].append(tickers[i])
if VERBOSE:
    # Print clustered tickers
    print("Clustered tickers:")
    for label, tickers in label_dict.items():
        print(f"{label}: {', '.join(tickers)}")

    # Count number of clusters for each size
    cluster_sizes = pd.Series([len(tickers) for tickers in label_dict.values()])
    print("\nCluster sizes:")
    print(cluster_sizes.value_counts().sort_index())   


Clustered tickers:
0: AKAM, CTVA, GEN, HST, IFF, PPG
1: CEG, GEV
2: CAG, CPB, GIS, HRL, HSY, KDP, KHC, MDLZ, MKC, MNST, PEP, SJM, TSN
3: AMGN, ELV, GILD
4: AMZN, EBAY, FTNT, META, PCAR, ROST, WDAY
5: EQR, IRM
6: ABT, ACN, BAX, BDX, BSX, ECL, HOLX, MCD, MDT, SYY, WMT, YUM, ZBH, ZTS
7: CSGP, DXCM
8: APO, BX, GS, KKR, MS
9: ANET, JBL
10: DG, TSCO
11: BEN, MGM, NUE
12: BKR, COP, CTRA, CVX, DVN, EOG, FANG, HAL, HES, OXY, SLB, XOM
13: MCHP, ON, SWKS, TER
14: EQT, EXE
15: ARE, BXP, PLD
16: CAH, CMCSA, CSCO, EA, JNJ, JNPR, K, KO, KR, MO, NEM, NWSA, PG, T, TMUS, VZ, WBA
17: AMCR, AOS, FDX, MAS, ODFL, SHW, UPS
18: BBY, HAS, HD, LOW, TGT
19: AME, BALL, CARR, CAT, CBRE, CSX, CTSH, DIS, ETN, F, GLW, GM, IR, IVZ, JCI, JPM, NFLX, PYPL, TJX, TROW, UNP, XYL
20: ADM, BF-B, STZ, TAP
21: AXP, BAC, C, CFG, COF, FITB, HBAN, KEY, MTB, PNC, RF, SCHW, STT, SYF, TFC, USB, WFC
22: DD, FCX, PNR, SWK
23: ADBE, COO, CPRT, CRM, EW, GEHC, GPN
24: WSM
25: GE, HWM
26: KMI, OKE, TRGP, WMB
27: CI, CNC, CVS, UNH
28: CZR, 

In [273]:
# Get two most similar tickers from each cluster with more than 1 ticker
similar_tickers = {}
for label, tickers in label_dict.items():
    if len(tickers) > 1:
        # Calculate pairwise correlation matrix for tickers in the cluster
        cluster_data = clean_data[tickers]
        corr_matrix = cluster_data.corr()
        
        # Get the two most similar tickers based on correlation
        most_similar = corr_matrix.unstack().sort_values(ascending=False)
        most_similar = most_similar[most_similar < 1]  # Exclude self-correlations
        top_pairs = most_similar.head(2).index.tolist()
        
        similar_tickers[label] = top_pairs
# Print the most similar tickers for each cluster
print("\nMost similar tickers in each cluster:")
for label, pairs in similar_tickers.items():
    print(f"Cluster {label}: {pairs[0]}")


Most similar tickers in each cluster:
Cluster 0: ('GEN', 'PPG')
Cluster 1: ('CEG', 'GEV')
Cluster 2: ('MKC', 'CPB')
Cluster 3: ('ELV', 'GILD')
Cluster 4: ('WDAY', 'META')
Cluster 5: ('EQR', 'IRM')
Cluster 6: ('YUM', 'WMT')
Cluster 7: ('CSGP', 'DXCM')
Cluster 8: ('BX', 'KKR')
Cluster 9: ('ANET', 'JBL')
Cluster 10: ('DG', 'TSCO')
Cluster 11: ('BEN', 'MGM')
Cluster 12: ('HAL', 'SLB')
Cluster 13: ('MCHP', 'TER')
Cluster 14: ('EQT', 'EXE')
Cluster 15: ('BXP', 'PLD')
Cluster 16: ('T', 'VZ')
Cluster 17: ('ODFL', 'MAS')
Cluster 18: ('HD', 'LOW')
Cluster 19: ('ETN', 'JCI')
Cluster 20: ('BF-B', 'TAP')
Cluster 21: ('FITB', 'USB')
Cluster 22: ('DD', 'SWK')
Cluster 23: ('ADBE', 'COO')
Cluster 25: ('GE', 'HWM')
Cluster 26: ('KMI', 'OKE')
Cluster 27: ('CNC', 'UNH')
Cluster 28: ('CZR', 'WYNN')
Cluster 29: ('HON', 'OTIS')
Cluster 30: ('HIG', 'L')
Cluster 31: ('AEP', 'WEC')
Cluster 32: ('DHI', 'LEN')
Cluster 33: ('IPG', 'OMC')
Cluster 34: ('CL', 'CHD')
Cluster 35: ('DELL', 'HPQ')
Cluster 37: ('PSX', 'V